In [10]:
# --- 1. 환경 설정 셀 (수정된 버전) ---
import os
import sys

# 작업 디렉토리를 올바른 위치로 변경
workspace_path = '/workspace'
os.chdir(workspace_path)

# RFdiffusion 경로를 파이썬이 인식하도록 추가
rfdiffusion_path = os.path.join(workspace_path, 'RFdiffusion')
if rfdiffusion_path not in sys.path:
    sys.path.append(rfdiffusion_path)
import time
import signal
import sys
import random
import string
import re
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
import py3Dmol

# Colab이 아닌 로컬 환경이므로, RFdiffusion 경로를 직접 추가
if 'RFdiffusion' not in sys.path:
    home_dir = os.path.expanduser("~")
    rfdiffusion_path = os.path.join(home_dir, 'RFdiffusion')
    sys.path.append(rfdiffusion_path)

os.environ["DGLBACKEND"] = "pytorch"

from inference.utils import parse_pdb
from colabdesign.rf.utils import get_ca
from colabdesign.rf.utils import fix_contigs, fix_partial_contigs, fix_pdb, sym_it
from colabdesign.shared.protein import pdb_to_string
from colabdesign.shared.plot import plot_pseudo_3D

# --- Colab의 files.upload()를 대체하는 로컬 파일 처리 함수 ---
def get_pdb(pdb_code=None, use_upload=False):
    if use_upload:
        upload_widget = widgets.FileUpload(
            accept='.pdb', description='PDB 파일 업로드', button_style='info'
        )
        display(upload_widget)
        
        def wait_for_upload(widget):
            while len(widget.value) == 0: time.sleep(0.1)
            uploaded_filename = list(widget.value.keys())[0]
            pdb_content = widget.value[uploaded_filename]['content']
            pdb_filename = "tmp.pdb"
            with open(pdb_filename, "wb") as out: out.write(pdb_content)
            print(f"'{uploaded_filename}'이(가) 'tmp.pdb'로 저장되었습니다.")
            return pdb_filename
            
        return wait_for_upload(upload_widget)
    elif pdb_code is None or pdb_code == "":
        print("PDB 코드를 입력하거나 use_upload=True로 설정하세요.")
        return None
    elif os.path.isfile(pdb_code): return pdb_code
    elif len(pdb_code) == 4:
        pdb_filename = f"{pdb_code}.pdb1"
        if not os.path.isfile(pdb_filename):
            os.system(f"wget -qnc https://files.rcsb.org/download/{pdb_code}.pdb1.gz && gunzip -f {pdb_code}.pdb1.gz")
        return pdb_filename
    else:
        pdb_filename = f"AF-{pdb_code}-F1-model_v4.pdb"
        if not os.path.isfile(pdb_filename):
            os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/{pdb_filename}")
        return pdb_filename

def run_ananas(pdb_str, path, sym=None):
    pdb_filename = f"outputs/{path}/ananas_input.pdb"
    out_filename = f"outputs/{path}/ananas.json"
    os.makedirs(f"outputs/{path}", exist_ok=True)
    with open(pdb_filename,"w") as handle: handle.write(pdb_str)
    cmd = f"./ananas {pdb_filename} -u -j {out_filename}"
    if sym is None: os.system(cmd)
    else: os.system(f"{cmd} {sym}")
    try:
        with open(out_filename,"r") as f: out = json.load(f)
        results, AU = out[0], out[-1]["AU"]
        group, chains, rmsd = AU["group"], AU["chain names"], results["Average_RMSD"]
        print(f"AnAnaS detected {group} symmetry at RMSD:{rmsd:.3}")
        C = np.array(results['transforms'][0]['CENTER'])
        A = [np.array(t["AXIS"]) for t in results['transforms']]
        new_lines = []
        for line in pdb_str.split("\n"):
            if line.startswith("ATOM"):
                chain = line[21:22]
                if chain in chains:
                    x = np.array([float(line[i:(i+8)]) for i in [30,38,46]])
                    if group[0] == "c": x = sym_it(x,C,A[0])
                    if group[0] == "d": x = sym_it(x,C,A[1],A[0])
                    coord_str = "".join([f"{a:8.3f}" for a in x])
                    new_lines.append(line[:30]+coord_str+line[54:])
            else: new_lines.append(line)
        return results, "\n".join(new_lines)
    except Exception as e:
        print(f"AnAnaS 결과 처리 중 오류 발생: {e}")
        return None, pdb_str

def run(command, steps, num_designs=1, visual="none"):
    def run_command_and_get_pid(command):
        pid_file = '/dev/shm/pid'
        os.system(f'nohup {command} > /dev/null & echo $! > {pid_file}')
        with open(pid_file, 'r') as f: pid = int(f.read().strip())
        os.remove(pid_file)
        return pid
    def is_process_running(pid):
        try: os.kill(pid, 0)
        except OSError: return False
        else: return True
    run_output = widgets.Output()
    progress = widgets.FloatProgress(min=0, max=1, description='running', bar_style='info')
    display(widgets.VBox([progress, run_output]))
    for n in range(steps):
        if os.path.isfile(f"/dev/shm/{n}.pdb"): os.remove(f"/dev/shm/{n}.pdb")
    pid = run_command_and_get_pid(command)
    try:
        fail = False
        for _ in range(num_designs):
            for n in range(steps):
                wait = True
                while wait and not fail:
                    time.sleep(0.1)
                    if os.path.isfile(f"/dev/shm/{n}.pdb"):
                        with open(f"/dev/shm/{n}.pdb", "r") as f: pdb_str = f.read()
                        if "TER" in pdb_str or "ENDMDL" in pdb_str: wait = False
                        elif not is_process_running(pid): fail = True
                    elif not is_process_running(pid): fail = True
                if fail:
                    progress.bar_style = 'danger'; progress.description = "failed"
                    break
                else:
                    progress.value = (n+1) / steps
                    if visual != "none":
                        with run_output:
                            run_output.clear_output(wait=True)
                            if visual == "image":
                                xyz, bfact = get_ca(f"/dev/shm/{n}.pdb", get_bfact=True)
                                fig = plt.figure(); fig.set_dpi(100);fig.set_figwidth(6);fig.set_figheight(6)
                                ax1 = fig.add_subplot(111);ax1.set_xticks([]);ax1.set_yticks([])
                                plot_pseudo_3D(xyz, c=bfact, cmin=0.5, cmax=0.9, ax=ax1); plt.show()
                            if visual == "interactive":
                                view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
                                view.addModel(pdb_str,'pdb'); view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
                                view.zoomTo(); view.show()
                if os.path.exists(f"/dev/shm/{n}.pdb"): os.remove(f"/dev/shm/{n}.pdb")
            if fail:
                progress.bar_style = 'danger'; progress.description = "failed"
                break
        while is_process_running(pid): time.sleep(0.1)
    except KeyboardInterrupt:
        os.kill(pid, signal.SIGTERM); progress.bar_style = 'danger'; progress.description = "stopped"
    
def run_diffusion(contigs, path, pdb=None, iterations=50, symmetry="none", order=1, hotspot=None,
                  chains=None, add_potential=False, num_designs=1, visual="none"):
    full_path = f"outputs/{path}"; os.makedirs(full_path, exist_ok=True)
    opts = [f"inference.output_prefix={full_path}", f"inference.num_designs={num_designs}"]
    if chains == "": chains = None
    if symmetry in ["auto","cyclic","dihedral"]:
        if symmetry == "auto": sym, copies = None, 1
        else: sym, copies = {"cyclic":(f"c{order}",order), "dihedral":(f"d{order}",order*2)}[symmetry]
    else: symmetry = None; sym, copies = None, 1
    contigs_list = contigs.replace(","," ").replace(":"," ").split()
    is_fixed, is_free = False, False; fixed_chains = []
    for contig in contigs_list:
        for x in contig.split("/"):
            a = x.split("-")[0]
            if a and a[0].isalpha():
                is_fixed = True
                if a[0] not in fixed_chains: fixed_chains.append(a[0])
            if a.isnumeric(): is_free = True
    if len(contigs_list) == 0 or not is_free: mode = "partial"
    elif is_fixed: mode = "fixed"
    else: mode = "free"
    if mode in ["partial","fixed"]:
        pdb_str = pdb_to_string(pdb, chains=chains)
        if symmetry == "auto":
            a, pdb_str = run_ananas(pdb_str, path)
            if a is None: print(f'ERROR: no symmetry detected'); symmetry = None; sym, copies = None, 1
            else:
                if a["group"][0] == "c": symmetry = "cyclic"; sym, copies = a["group"], int(a["group"][1:])
                elif a["group"][0] == "d": symmetry = "dihedral"; sym, copies = a["group"], 2 * int(a["group"][1:])
                else: print(f'ERROR: detected symm ({a["group"]}) not supported'); symmetry = None; sym, copies = None, 1
        elif mode == "fixed": pdb_str = pdb_to_string(pdb_str, chains=fixed_chains)
        pdb_filename = f"{full_path}/input.pdb"
        with open(pdb_filename, "w") as handle: handle.write(pdb_str)
        parsed_pdb = parse_pdb(pdb_filename)
        opts.append(f"inference.input_pdb={pdb_filename}")
        if mode in ["partial"]:
            iterations = int(80 * (iterations / 200)); opts.append(f"diffuser.partial_T={iterations}")
            contigs_list = fix_partial_contigs(contigs_list, parsed_pdb)
        else: opts.append(f"diffuser.T={iterations}"); contigs_list = fix_contigs(contigs_list, parsed_pdb)
    else:
        opts.append(f"diffuser.T={iterations}"); parsed_pdb = None
        contigs_list = fix_contigs(contigs_list, parsed_pdb)
    if hotspot is not None and hotspot != "": opts.append(f"ppi.hotspot_res=[{hotspot}]")
    if sym is not None:
        sym_opts = ["--config-name symmetry", f"inference.symmetry={sym}"]
        if add_potential: sym_opts += ["'potentials.guiding_potentials=[\"type:olig_contacts,weight_intra:1,weight_inter:0.1\"]'", "potentials.olig_intra_all=True","potentials.olig_inter_all=True", "potentials.guide_scale=2","potentials.guide_decay=quadratic"]
        opts = sym_opts + opts; contigs_list = sum([contigs_list] * copies,[])
    opts.append(f"'contigmap.contigs=[{' '.join(contigs_list)}]'")
    opts += ["inference.dump_pdb=True","inference.dump_pdb_path='/dev/shm'"]
    print("mode:", mode); print("output:", full_path); print("contigs:", contigs_list)
    run_script_path = "/workspace/RFdiffusion/run_inference.py"
    opts_str = " ".join(opts)
    cmd = f"python {run_script_path} {opts_str}"
    print(cmd)
    run(cmd, iterations, num_designs, visual=visual)
    for n in range(num_designs):
        pdbs = [f"outputs/traj/{path}_{n}_pX0_traj.pdb", f"outputs/traj/{path}_{n}_Xt-1_traj.pdb", f"{full_path}_{n}.pdb"]
        for pdb_file in pdbs:
            if os.path.exists(pdb_file):
                with open(pdb_file,"r") as handle: pdb_str = handle.read()
                with open(pdb_file,"w") as handle: handle.write(fix_pdb(pdb_str, contigs_list))
    return contigs_list, copies

print("✅ Setup cell is ready.")

✅ Setup cell is ready.


In [1]:
%%time
import os
import sys
import pandas as pd
import glob

# --- 1. 설정 및 파라미터 입력 부분 (첫 번째 코드) ---

# @markdown ### **👇 여기에 처리할 폴더와 Contig 정보를 입력하세요.**
# @markdown ---
input_folder = "outputs/human-best"  # @param {type:"string"}
contig_for_all_pdbs = "C311-391/0 100-100"  # @param {type:"string"}

# @markdown ---
# @markdown ### **MPNN 및 AlphaFold 공통 설정**
# @markdown 이 설정은 위 폴더의 모든 PDB 파일에 동일하게 적용됩니다.
num_seqs = 8       #@param {type:"raw"}
num_recycles = 3    #@param {type:"raw"}
initial_guess = True
use_multimer = True
rm_aa = "C"
mpnn_sampling_temp = 0.1

# --- 2. 개별 PDB 파일 처리 루프 (첫 번째 코드) ---

print(f"🚀 Settings: num_seqs={num_seqs}, num_recycles={num_recycles}")
pdb_files = sorted(glob.glob(os.path.join(input_folder, '*.pdb')))

all_results_list = [] # 모든 결과를 저장할 리스트

if not pdb_files:
    print(f"🚨 에러: '{input_folder}' 폴더에서 PDB 파일을 찾을 수 없습니다.")
else:
    print(f"총 {len(pdb_files)}개의 PDB 파일을 대상으로 작업을 시작합니다.")
    print("="*60)

    for i, pdb_path in enumerate(pdb_files):
        print(f"\n({i+1}/{len(pdb_files)}) 번째 작업 실행 중...")
        print(f"  - 입력 파일: {pdb_path}")

        pdb_basename = os.path.basename(pdb_path).replace('.pdb', '')
        output_dir_name = f"{pdb_basename}_mpnn_results"
        output_loc = os.path.join(input_folder, output_dir_name)
        os.makedirs(output_loc, exist_ok=True)
        print(f"  - 결과 저장: {output_loc}")

        contigs_str = ":".join(contig_for_all_pdbs.replace("'", "").replace('"', '').split())
        opts = [f"--pdb={pdb_path}", f"--loc={output_loc}", f"--contig={contigs_str}",
                f"--copies=1", f"--num_seqs={num_seqs}", f"--num_recycles={num_recycles}",
                f"--rm_aa={rm_aa}", f"--mpnn_sampling_temp={mpnn_sampling_temp}", f"--num_designs=1"]
        if initial_guess: opts.append("--initial_guess")
        if use_multimer: opts.append("--use_multimer")
        opts_str = ' '.join(opts)

        # ColabDesign 실행
        get_ipython().system(f"python -m colabdesign.rf.designability_test {opts_str}")
        
        results_csv_path = f"{output_loc}/mpnn_results.csv"
        
        if os.path.exists(results_csv_path):
            df = pd.read_csv(results_csv_path)
            best_row = df.sort_values(by=['design', 'n'], ascending=True).iloc[0]
            rmsd_val = best_row['rmsd']
            plddt_val = best_row['plddt']
            print(f"  ✅ 완료! Best RMSD: {rmsd_val:.4f}, pLDDT: {plddt_val:.2f}")

            # 나중에 합치기 위해 원본 파일명 열 추가
            df['source_pdb'] = os.path.basename(pdb_path)
            all_results_list.append(df)
        else:
            print(f"  ⚠️ 경고: 최종 결과 파일({results_csv_path})이 생성되지 않았습니다.")
        print("-" * 60)

# --- 3. 최종 결과 취합 및 요약 (두 번째 코드) ---

print("\n🎉 모든 개별 작업이 완료되었습니다. 전체 결과를 요약합니다.")
print("="*60)

if all_results_list:
    final_df = pd.concat(all_results_list, ignore_index=True)
    final_df_sorted = final_df.sort_values(by="rmsd", ascending=True)

    # --- ✨ 최종 종합 결과 파일로 저장 ✨ ---
    # (1) 최종 CSV 파일로 저장
    final_csv_path = os.path.join(input_folder, "mpnn_results.csv")
    final_df_sorted.to_csv(final_csv_path, index=False)
    print(f"✅ 최종 종합 결과가 CSV 파일로 저장되었습니다: {final_csv_path}")

    print("\n📊 **전체 디자인 상세 결과 (RMSD 오름차순)**")
    
    # i_pae가 있는지 확인하고 출력할 컬럼 결정
    display_cols = ['source_pdb', 'n', 'plddt', 'rmsd']
    if 'i_pae' in final_df_sorted.columns:
        display_cols.insert(3, 'i_pae')
    
    # .to_string()을 이용해 전체 데이터프레임 출력
    print(final_df_sorted[display_cols].to_string(index=False))
    
    print("-" * 60)
    overall_best = final_df_sorted.iloc[0]
    print(f"🏆 **최종 BEST 디자인** 🏆")
    print(f"  - 원본 백본 파일: {overall_best['source_pdb']}")
    print(f"  - 시퀀스 번호 (n): {overall_best['n']}")
    print(f"  - **최종 RMSD: {overall_best['rmsd']:.4f}**")
    print(f"  - 최종 pLDDT: {overall_best['plddt']:.2f}")
    
    # i_pae 값이 존재하고 NaN이 아닐 경우에만 출력
    if 'i_pae' in overall_best and not pd.isna(overall_best['i_pae']):
        print(f"  - 최종 i_pae: {overall_best['i_pae']:.2f}")
    
    best_pdb_loc_name = overall_best['source_pdb'].replace('.pdb','_mpnn_results')
    best_pdb_loc_path = os.path.join(input_folder, best_pdb_loc_name)
    print(f"\n  👉 이 BEST 디자인의 PDB 파일은 '{best_pdb_loc_path}/best.pdb' 에서 확인할 수 있습니다.")

else:
    print("처리된 결과가 없어 최종 요약을 생성할 수 없습니다.")

🚀 Settings: num_seqs=8, num_recycles=3
총 22개의 PDB 파일을 대상으로 작업을 시작합니다.

(1/22) 번째 작업 실행 중...
  - 입력 파일: outputs/human-best/0806-150-300_5m3ce_168.pdb
  - 결과 저장: outputs/human-best/0806-150-300_5m3ce_168_mpnn_results
{'pdb':'outputs/human-best/0806-150-300_5m3ce_168.pdb','loc':'outputs/human-best/0806-150-300_5m3ce_168_mpnn_results','contigs':'C311-391/0:100-100','copies':1,'num_seqs':8,'initial_guess':True,'use_multimer':True,'use_soluble':False,'num_recycles':3,'rm_aa':'C','num_designs':1,'mpnn_sampling_temp':0.1}
protocol=binder
2025-08-19 07:24:21.506171: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.8 which is older than the ptxas CUDA version (12.9.86). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
running proteinMPNN...
running AlphaFold...
design:0 n:0 mpnn:

design:0 n:5 mpnn:1.149 plddt:0.900 i_ptm:0.550 i_pae:12.336 rmsd:2.888 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/GVKPEELKKFTEELIKRRKERTKKENEEEREKMKKEFEKKLKEELAKAPPEEHEKVLKEAEKEKKELEKKFEEEAKKNEERTKKINEEIKKNTEKRIKSL
design:0 n:6 mpnn:1.181 plddt:0.788 i_ptm:0.073 i_pae:26.230 rmsd:45.750 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/GVKPEELKEETEKLIKKKEKKTEEEAKKEKEKLEKEFAEKLAARLAAAPASKHAEVKAEAAKEKAALEEEFKKKEEERKKQTKKINEEIEKNTNKYIESL
design:0 n:7 mpnn:1.146 plddt:0.864 i_ptm:0.521 i_pae:13.362 rmsd:4.429 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/GVKKEELEEDTKKLIERKKKKTEEEAKKEEEKLKKEFEKRYKEKLAAAPPEKHEEVEKEFKKEEEKLKKELEKEKEKRKKRTEEINKEIKENTKKRIEEI
  ✅ 완료! Best RMSD: 39.2617, pLDDT: 0.84
------------------------------------------------------------

(4/22) 번째 작업 실행 중...
  - 입력 파일: outputs/human-best/0807-150-600_418.pdb
  - 결과 저장: outputs/human-best/0807-150-600_418_mpnn_results


design:0 n:2 mpnn:1.180 plddt:0.907 i_ptm:0.057 i_pae:28.352 rmsd:62.202 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/EEEEKRKKEEERKEEERKRREEAAKQNEENEKRRKEKKEEEEKKKKELEEKRKKLEEEAEKTKKELEEMEKNKEEYSEEEIKKKKEELEKLKAEIEKLKK
design:0 n:3 mpnn:1.138 plddt:0.897 i_ptm:0.284 i_pae:20.926 rmsd:2.262 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/EEEERRRKEEEREKEEEERRKQAEEQNRRNEERIKEEEKKKKEEEEKKKKELEKLKKEAEELKKKLEEMKEKKEEYSEEEIEEVEKKLEELKKKIEELEK
design:0 n:4 mpnn:1.157 plddt:0.788 i_ptm:0.561 i_pae:16.893 rmsd:5.709 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/KEKEEEEKERKREEEERRRREEARKQNERNEKLKKEREAKEKEEKEKLEKEREELEKKAKEIEKKLEEFEKKKEEYSEEEIEKLKEELEKLKKKIEELKK
design:0 n:5 mpnn:1.148 plddt:0.903 i_ptm:0.562 i_pae:14.663 rmsd:4.401 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SLEEEERRRRKREEEEKERRKKAEEQNKKNEELKKKKKEEEEKKKEELEKKKKELEKKYEELKKKLEEMEKNKEKYSEEE

running proteinMPNN...
running AlphaFold...
design:0 n:0 mpnn:1.140 plddt:0.830 i_ptm:0.111 i_pae:24.119 rmsd:33.414 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SEEEVKEIGKKIEEKEKELQAEWKAIDAELAKLDPATAAAIRAAIDRLEATGWDDPEYAALRDAYRAAGNEAAAAILERARERIKKREAFREELKKELKE
design:0 n:1 mpnn:1.176 plddt:0.818 i_ptm:0.102 i_pae:24.644 rmsd:32.529 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SEEEVKEIGKEIESKQKELQKKWEEIKKELEKFSPEERKEIEELMEKMEEEGYDSEEYEKARKKYKEEGNEEAAEVLEKAREREKERKEFRKKLKKKLKE
design:0 n:2 mpnn:1.165 plddt:0.849 i_ptm:0.107 i_pae:24.157 rmsd:27.376 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SEEEVKELSKEIEKKRKELQEKWKEIKEELAKLSPEEKKKVEAAIEEMEATGWESPKYEALAKEYEEKGNKKAAEILKKARERIKEREEFRKKLRKKLKE
design:0 n:3 mpnn:1.152 plddt:0.820 i_ptm:0.286 i_pae:19.193 rmsd:3.979 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SKEEIKERSKEIEKKQKELSEKSKEIKEELAKLS

{'pdb':'outputs/human-best/0811-150-200_24.pdb','loc':'outputs/human-best/0811-150-200_24_mpnn_results','contigs':'C311-391/0:100-100','copies':1,'num_seqs':8,'initial_guess':True,'use_multimer':True,'use_soluble':False,'num_recycles':3,'rm_aa':'C','num_designs':1,'mpnn_sampling_temp':0.1}
protocol=binder
2025-08-19 07:35:27.874048: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.8 which is older than the ptxas CUDA version (12.9.86). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
running proteinMPNN...
running AlphaFold...
design:0 n:0 mpnn:1.069 plddt:0.865 i_ptm:0.074 i_pae:26.485 rmsd:21.649 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/ELEKKKKEIKELLNKYTKLKNEGGDEEEAKKALDEAKKLLEELKPGGAELLKKYEEKKEEVDKLKKEYEKTKDAEIKNKYLEAVKELEKLKE

design:0 n:6 mpnn:1.085 plddt:0.912 i_ptm:0.531 i_pae:11.341 rmsd:4.027 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/LEKKADELLDETFKLKEEMEKEVAAAKKEAAALDPETAKKKLKETLEKYAKTLEEIAEKYKEIAKLEKELGNEKMAKLYEKAAEQTKKEAEQVKAKAAAL
design:0 n:7 mpnn:1.025 plddt:0.894 i_ptm:0.545 i_pae:11.423 rmsd:2.974 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/LEKEAEKLLDEYKKKKKELEEEVKKAEEEAAKLSPEEKKKKLKEMLEKKAEKLEELAEMLEKIAELEKKLGNEKMAKLYEKAAKQIKKEAEKLKKKAEEL
  ✅ 완료! Best RMSD: 11.0377, pLDDT: 0.87
------------------------------------------------------------

(15/22) 번째 작업 실행 중...
  - 입력 파일: outputs/human-best/0811-200-200_a1168_142.pdb
  - 결과 저장: outputs/human-best/0811-200-200_a1168_142_mpnn_results
{'pdb':'outputs/human-best/0811-200-200_a1168_142.pdb','loc':'outputs/human-best/0811-200-200_a1168_142_mpnn_results','contigs':'C311-391/0:100-100','copies':1,'num_seqs':8,'initial_guess':True,'use_multimer':True,'use_soluble':False,'num_recy

design:0 n:3 mpnn:0.976 plddt:0.916 i_ptm:0.493 i_pae:14.077 rmsd:3.091 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/MTKEELREKRLEAKILKNKALELKNEAKKKLEKAEELEKKAKELKKKGEEEKAKELEKKAKELRKEAEKLLSTEVGKLNEEAEKIEKEVEEELKKRKEEE
design:0 n:4 mpnn:0.987 plddt:0.893 i_ptm:0.136 i_pae:23.520 rmsd:5.201 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/MTPEELREKELKAKILKNKAQELKNEADKKLKEAEKLKAEAEKLKKEGKEEEAAKLKKEAEELKAKAKETLETKVKKLEEEAKKILEEVEAEKKAREEAK
design:0 n:5 mpnn:0.970 plddt:0.904 i_ptm:0.454 i_pae:15.364 rmsd:1.424 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/MTKEELREKRIKASILKNKALELKNEAKKKLEEAKKLKAEAEKLKKEGKEEEAAKLEAEAAKKEAEAEKVLKEKVKKLEEEAKKILEEVKEEEKKRKEEK
design:0 n:6 mpnn:0.965 plddt:0.906 i_ptm:0.077 i_pae:26.372 rmsd:15.335 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/MSKEELREKRLEASKLKNKALELENKAKKILAEADKLEAEAKALAKEGKEEEAAKLKKEAAAKRAEAKKILEEEVKSLKE

running proteinMPNN...
running AlphaFold...
design:0 n:0 mpnn:1.081 plddt:0.673 i_ptm:0.097 i_pae:24.669 rmsd:30.796 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/MEEKKKEEEKRKKIAETSKKINKLLNETRKKVEELLKKGKEEEALKEMEKAKKEIKKLFEELEKVVDKNLAEKRLKIREKELEEELEELKKRIEKKRKEE
design:0 n:1 mpnn:1.134 plddt:0.699 i_ptm:0.068 i_pae:26.668 rmsd:32.037 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SSLLKKKEEQRKKIAETRKKINEILNKTRKEVEKLEKEGKYEEALKKIEEALEKIEKIYEELKKVVSEELAEKMLEREKKRLEKEKKELEERIEKKKKEE
design:0 n:2 mpnn:1.056 plddt:0.711 i_ptm:0.088 i_pae:24.954 rmsd:29.080 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SEEKKKEEEKRKFIAETRKKVNELLNETRKKVEELLKEGKYEEAIKVIEETKKKIEELFKELEKKVDKELAEKMLKIYKERLEEEKEKLKKEIEEKKKKE
design:0 n:3 mpnn:1.116 plddt:0.830 i_ptm:0.069 i_pae:26.398 rmsd:37.137 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SEEEKKKKEQREAIAKTREEVNKLLNETDKKVK